<a href="https://colab.research.google.com/github/MiguelSix/AlgoritmoLineal/blob/master/limpieza_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONFIGURACION DE RUTA Y DATASET


In [ ]:
#MONTAR DRIVE COMPARTIDO

from google.colab import drive
import os

drive.mount('/content/drive')

shared_folder_id = "1ROZfrhsFxeazxoewZVlCydXeMTqQbdLw"

!gdown --folder --id {shared_folder_id}

#!ls -la

Mounted at /content/drive
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder contents
Processing file 1FPWs9Wd4Z1G2KDeA3N46luxOFKwfJ6He ASEGURADO.csv
Processing file 1c7eg6e0Q8bL1U6pjrNOjiL9FsVPAN1n6 Aseguradora
Processing file 10puKqkdp7nmBa3DJkbgo6lyNILIfPQhy Dataset.csv
Processing file 1v9xwPresWaRaZ9X-o-I-ohxYHuobDtWLk44Nszn3Qo4 Limpieza de dataset
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1FPWs9Wd4Z1G2KDeA3N46luxOFKwfJ6He
From (redirected): https://drive.google.com/uc?id=1FPWs9Wd4Z1G2KDeA3N46luxOFKwfJ6He&confirm=t&uuid=fda7559f-588b-4b74-9d7d-5081703edb9f
To: /content/Qualitas/ASEGURADO.csv
100% 633M/633M [00:04<00:00, 138MB/s]
Downloading...
From: https://drive.google.com

In [ ]:
#RUTA/CARGAR ARCHIVO
import pandas as pd

file_path = './Qualitas/ASEGURADO.csv'

df_principal = pd.read_csv(file_path)
print(df_principal.head())

                                  ASEG              SERIE    TELEFONO  \
0              REY DAVID BELMAN LOZANO  9BWKL45U6RP023988  4444191129   
1                  DAVILA MORENO CELSO  3G1TA5AF7HL120346  5560767580   
2        GARCIA GONZALEZ JESUS EDUARDO  1C4GJ25R23B244434  4611366507   
3  SD INDEVAL INSTITUCION PARA EL DEPO  JM3KKKHC2R1166032         NaN   
4           MARIA SEDELIA VARGAS ROSAS  MB2C22AC8JM241475  7531117734   

           DIR_ASEG_CALLE             COLONIA  \
0      CIRCUITO JACARANDA           EL ZAPOTE   
1  1RA CDA FERROCARRIL 12  SAN JUAN TEMAMATLA   
2    PROLONGACION CONSEJO             METEPEC   
3     PASEO DE LA REFORMA          CUAUHTEMOC   
4       LOC EL PUERTECITO           EL REPARO   

                                   POBL            ESTADO       CP  \
0       035 SOLEDAD DE GRACIANO SANCHEZ   SAN LUIS POTOSI  78430.0   
1                         083 TEMAMATLA  ESTADO DE MEXICO  56650.0   
2                           019 ATLIXCO            PUEBL

# ***CODIGOS DE LIMPIEZA DE MIGUEL***

In [ ]:
import unicodedata
import re

##***Columna del número de serie (SERIE)***

**Función para revisar la columna y checar los registros erroneos**

In [ ]:
def analizar_columna_serie(df, columna='SERIE'):

    def _verificar_serie(valor):
        # Manejar valores nulos o vacíos
        if pd.isna(valor) or valor == '':
            return {
                'original': valor,
                'valido': True,
                'problemas': []
            }

        # Convertir a string para asegurar consistencia
        valor = str(valor)

        problemas = []

        # Verificar espacios
        if valor != valor.strip():
            problemas.append('Espacios al inicio/final')

        # Verificar espacios entre caracteres
        if ' ' in valor:
            problemas.append('Espacios entre caracteres')

        # Verificar mayúsculas
        if valor != valor.upper():
            problemas.append('No está en mayúsculas')

        # Verificar caracteres especiales y acentos
        # Primero, remover acentos
        valor_sin_acentos = ''.join(
            c for c in unicodedata.normalize('NFD', valor)
            if unicodedata.category(c) != 'Mn'
        )

        # Verificar solo alfanuméricos
        if not re.match(r'^[A-Z0-9]+$', valor_sin_acentos):
            problemas.append('Contiene caracteres especiales o acentos')

        return {
            'original': valor,
            'valido': len(problemas) == 0,
            'problemas': problemas
        }

    # Aplicar análisis a toda la columna
    analisis = df[columna].apply(_verificar_serie)

    # Resumen estadístico
    resumen = {
        'total_registros': len(df),
        'registros_validos': sum(1 for x in analisis if x['valido']),
        'registros_invalidos': sum(1 for x in analisis if not x['valido']),
        'tipos_de_problemas': {}
    }

    # Contar tipos de problemas
    for item in analisis:
        if not item['valido']:
            for problema in item['problemas']:
                resumen['tipos_de_problemas'][problema] = resumen['tipos_de_problemas'].get(problema, 0) + 1

    return {
        'resumen': resumen,
        'detalles': analisis
    }

**Función para limpiar la columna SERIE (Corregir los errores)**

In [ ]:
def limpiar_columna_serie(df, columna='SERIE'):

    def _limpiar_serie(valor):
        # Manejar valores nulos o vacíos
        if pd.isna(valor) or valor == '':
            return ''

        # Convertir a string
        valor = str(valor)

        # Eliminar acentos
        valor_sin_acentos = ''.join(
            c for c in unicodedata.normalize('NFD', valor)
            if unicodedata.category(c) != 'Mn'
        )

        # Convertir a mayúsculas y eliminar espacios
        valor_limpio = valor_sin_acentos.upper().replace(' ', '')

        # Verificar que solo contenga letras y números
        if re.match(r'^[A-Z0-9]+$', valor_limpio):
            return valor_limpio

        # Si no cumple, devolver cadena vacía
        return ''

    # Crear una copia del DataFrame para no modificar el original
    df_limpio = df.copy()

    # Aplicar limpieza
    df_limpio[columna] = df_limpio[columna].apply(_limpiar_serie)

    return df_limpio

##***Columna del número de telefono (TELEFONO)***

**Función para analizar la columna**

In [ ]:
def analizar_telefonos(df, columna='TELEFONO'):
    # Convertir a string y crear copia para análisis
    serie = df[columna].astype(str)
    total_registros = len(serie)

    # Inicializar contadores
    analisis = {
        'total_registros': total_registros,
        'registros_vacios': 0,
        'mas_10_digitos': 0,
        'menos_10_digitos': 0,
        'tiene_simbolos': 0,
        'tiene_espacios_final': 0,
        'tiene_letras': 0,
        'ejemplos_problematicos': []
    }

    patrones = {
        'mas_10_digitos': [],
        'menos_10_digitos': [],
        'simbolos': [],
        'espacios_final': [],
        'letras': []
    }

    for valor in serie:
        # Saltear valores nulos o 'nan'
        if pd.isna(valor) or valor == 'nan':
            analisis['registros_vacios'] += 1
            continue

        # Verificar longitud de dígitos
        digitos = re.findall(r'\d', valor)
        if len(digitos) > 10:
            analisis['mas_10_digitos'] += 1
            if len(patrones['mas_10_digitos']) < 3:
                patrones['mas_10_digitos'].append(valor)

        # Verificar menos de 10 dígitos
        if len(digitos) < 10:
            analisis['menos_10_digitos'] += 1
            if len(patrones['menos_10_digitos']) < 3:
                patrones['menos_10_digitos'].append(valor)

        # Verificar símbolos
        if re.search(r'[+()/-]', valor):
            analisis['tiene_simbolos'] += 1
            if len(patrones['simbolos']) < 3:
                patrones['simbolos'].append(valor)

        # Verificar espacios al final
        if valor.endswith(' '):
            analisis['tiene_espacios_final'] += 1
            if len(patrones['espacios_final']) < 3:
                patrones['espacios_final'].append(valor)

        # Verificar letras
        if re.search(r'[a-zA-Z]', valor):
            analisis['tiene_letras'] += 1
            if len(patrones['letras']) < 3:
                patrones['letras'].append(valor)

    # Calcular porcentajes
    for key in ['registros_vacios', 'mas_10_digitos', 'menos_10_digitos', 'tiene_simbolos', 'tiene_espacios_final', 'tiene_letras']:
        porcentaje = (analisis[key] / total_registros) * 100
        print(f"{key}: {analisis[key]} registros ({porcentaje:.2f}%)")

    print("\nEjemplos de registros problemáticos:")
    for tipo, ejemplos in patrones.items():
        if ejemplos:
            print(f"\n{tipo}:")
            for ejemplo in ejemplos:
                print(f"  - {ejemplo}")

**Funcion para limpiar la columna (corregir errores)**

In [ ]:
def limpiar_telefonos(df, columna='TELEFONO'):
    # Convertir a string
    df[columna] = df[columna].astype(str)

    # Función para limpiar cada valor
    def limpiar_valor(valor):
        if pd.isna(valor) or valor == 'nan':
            return ''

        # Extraer solo los números usando expresiones regulares
        numeros = re.sub(r'[^0-9]', '', valor)

        # Si tiene menos de 7 cifras, devolver vacío
        return numeros if len(numeros) >= 7 else ''

    # Aplicar la limpieza
    df[columna] = df[columna].apply(limpiar_valor)

    # Verificar los resultados
    muestra = df[columna].head(10)
    print("\nMuestra de los primeros 10 registros limpiados:")
    for idx, valor in muestra.items():
        print(f"Original: {data[columna][idx]} -> Limpio: {valor}")

    return df

##***Columna del la calle (DIR_ASEG_CALLE)***

**Función para analizar la columna**

In [ ]:
import pandas as pd
import re

def analizar_calle(df, columna='DIR_ASEG_CALLE'):
    serie = df[columna].astype(str)
    total_registros = len(serie)

    analisis = {
        'total_registros': total_registros,
        'registros_vacios': 0,
        'multiples_espacios': 0,
        'tiene_minusculas': 0,
        'espacio_final': 0,
        'caracteres_especiales': 0,
        'comienza_con_numero': 0,
        'comienza_con_punto': 0,
        'parentesis_invalidos': 0,
        'guiones_invalidos': 0,
        'signos_plus_invalidos': 0,
        'dos_puntos_invalidos': 0
    }

    patrones = {
        'multiples_espacios': [],
        'minusculas': [],
        'espacio_final': [],
        'caracteres_especiales': [],
        'comienza_con_numero': [],
        'comienza_con_punto': [],
        'parentesis_invalidos': [],
        'guiones_invalidos': [],
        'signos_plus_invalidos': [],
        'dos_puntos_invalidos': []
    }

    for valor in serie:
        if pd.isna(valor) or valor.strip() == '' or valor == 'nan':
            analisis['registros_vacios'] += 1
            continue

        if '  ' in valor:
            analisis['multiples_espacios'] += 1
            if len(patrones['multiples_espacios']) < 3:
                patrones['multiples_espacios'].append(valor)

        if any(c.islower() for c in valor):
            analisis['tiene_minusculas'] += 1
            if len(patrones['minusculas']) < 3:
                patrones['minusculas'].append(valor)

        if valor.endswith(' '):
            analisis['espacio_final'] += 1
            if len(patrones['espacio_final']) < 3:
                patrones['espacio_final'].append(valor)

        if re.search(r'[^a-zA-Z0-9 .\(\)-:\+]', valor):
            analisis['caracteres_especiales'] += 1
            if len(patrones['caracteres_especiales']) < 3:
                patrones['caracteres_especiales'].append(valor)

        if re.match(r'^\d', valor) and not valor.replace(' ', '').isdigit():
            analisis['comienza_con_numero'] += 1
            if len(patrones['comienza_con_numero']) < 3:
                patrones['comienza_con_numero'].append(valor)

        if re.match(r'^\.', valor):
            analisis['comienza_con_punto'] += 1
            if len(patrones['comienza_con_punto']) < 3:
                patrones['comienza_con_punto'].append(valor)

        if re.match(r'^\(', valor):
            analisis['parentesis_invalidos'] += 1
            if len(patrones['parentesis_invalidos']) < 3:
                patrones['parentesis_invalidos'].append(valor)

        if re.match(r'^-', valor):
            analisis['guiones_invalidos'] += 1
            if len(patrones['guiones_invalidos']) < 3:
                patrones['guiones_invalidos'].append(valor)

        # Validate plus sign (not at the end)
        if re.search(r'\+$', valor):
            analisis['signos_plus_invalidos'] += 1
            if len(patrones['signos_plus_invalidos']) < 3:
                patrones['signos_plus_invalidos'].append(valor)

        if re.search(r':$', valor):
            analisis['dos_puntos_invalidos'] += 1
            if len(patrones['dos_puntos_invalidos']) < 3:
                patrones['dos_puntos_invalidos'].append(valor)

    print(f"\nAnálisis de la columna '{columna}':")
    print(f"Total de registros: {total_registros}")

    for key in analisis.keys():
        if key != 'total_registros':
            porcentaje = (analisis[key] / total_registros) * 100
            print(f"{key}: {analisis[key]} registros ({porcentaje:.2f}%)")

    print("\nEjemplos de registros problemáticos:")
    for tipo, ejemplos in patrones.items():
        if ejemplos:
            print(f"\n{tipo}:")
            for ejemplo in ejemplos:
                print(f"  - {ejemplo}")

    return analisis, patrones

**Función para limpiar los errores de la columna CALLE**

In [ ]:
def limpiar_direcciones(df, columna='DIR_ASEG_CALLE'):
    # Convertir a string y crear copia para análisis
    serie = df[columna].astype(str)

    # Función interna para limpiar un valor individual
    def limpiar_valor(valor):
        # Manejar valores nulos o vacíos
        if pd.isna(valor) or valor.strip() == '' or valor == 'nan':
            return ''

        # Si el valor contiene solo números (incluso con espacios), devolver cadena vacía
        if valor.replace(' ', '').isdigit():
            return ''

        # Eliminar espacio inicial y final
        valor = valor.strip()

        # Eliminar punto al inicio
        if valor.startswith('.'):
            valor = valor[1:]

        # Eliminar '+' o '-' al inicio
        if valor.startswith('+'):
            valor = valor[1:]

        if valor.startswith('-'):
            valor = valor[1:]

        # Eliminar todos los paréntesis
        valor = valor.replace('(', '').replace(')', '')

        # Eliminar signos al final: '+', '-', ':', '.'
        valor = re.sub(r'[+\-:.]$', '', valor)

        # Eliminar dobles espacios
        valor = re.sub(r'\s+', ' ', valor)

        # Convertir a mayúsculas
        valor = valor.upper()

        return valor.strip()

    # Aplicar limpieza a toda la serie
    serie_limpia = serie.apply(limpiar_valor)

    # Crear nuevo DataFrame con la columna limpia
    df_limpio = df.copy()
    df_limpio[columna] = serie_limpia

    return df_limpio

##***Columna del la colonia (COLONIA)***

**Función para analizar la columna**

In [ ]:
import pandas as pd
import re

def analizar_colonia(df, columna='COLONIA'):
    serie = df[columna].astype(str)
    total_registros = len(serie)

    analisis = {
        'total_registros': total_registros,
        'registros_vacios': 0,
        'multiples_espacios': 0,
        'tiene_minusculas': 0,
        'espacio_final': 0,
        'caracteres_especiales': 0,
        'comienza_con_numero': 0,
        'comienza_con_punto': 0,
        'parentesis_invalidos': 0,
        'guiones_invalidos': 0,
        'signos_plus_invalidos': 0,
        'dos_puntos_invalidos': 0
    }

    patrones = {
        'multiples_espacios': [],
        'minusculas': [],
        'espacio_final': [],
        'caracteres_especiales': [],
        'comienza_con_numero': [],
        'comienza_con_punto': [],
        'parentesis_invalidos': [],
        'guiones_invalidos': [],
        'signos_plus_invalidos': [],
        'dos_puntos_invalidos': []
    }

    for valor in serie:
        if pd.isna(valor) or valor.strip() == '' or valor == 'nan':
            analisis['registros_vacios'] += 1
            continue

        if '  ' in valor:
            analisis['multiples_espacios'] += 1
            if len(patrones['multiples_espacios']) < 3:
                patrones['multiples_espacios'].append(valor)

        if any(c.islower() for c in valor):
            analisis['tiene_minusculas'] += 1
            if len(patrones['minusculas']) < 3:
                patrones['minusculas'].append(valor)

        if valor.endswith(' '):
            analisis['espacio_final'] += 1
            if len(patrones['espacio_final']) < 3:
                patrones['espacio_final'].append(valor)

        if re.search(r'[^a-zA-Z0-9 .\(\)-:\+]', valor):
            analisis['caracteres_especiales'] += 1
            if len(patrones['caracteres_especiales']) < 3:
                patrones['caracteres_especiales'].append(valor)

        if re.match(r'^\d', valor) and not valor.replace(' ', '').isdigit():
            analisis['comienza_con_numero'] += 1
            if len(patrones['comienza_con_numero']) < 3:
                patrones['comienza_con_numero'].append(valor)

        if re.match(r'^\.', valor):
            analisis['comienza_con_punto'] += 1
            if len(patrones['comienza_con_punto']) < 3:
                patrones['comienza_con_punto'].append(valor)

        if re.match(r'^\(', valor):
            analisis['parentesis_invalidos'] += 1
            if len(patrones['parentesis_invalidos']) < 3:
                patrones['parentesis_invalidos'].append(valor)

        if re.match(r'^-', valor):
            analisis['guiones_invalidos'] += 1
            if len(patrones['guiones_invalidos']) < 3:
                patrones['guiones_invalidos'].append(valor)

        # Validate plus sign (not at the end)
        if re.search(r'\+$', valor):
            analisis['signos_plus_invalidos'] += 1
            if len(patrones['signos_plus_invalidos']) < 3:
                patrones['signos_plus_invalidos'].append(valor)

        if re.search(r':$', valor):
            analisis['dos_puntos_invalidos'] += 1
            if len(patrones['dos_puntos_invalidos']) < 3:
                patrones['dos_puntos_invalidos'].append(valor)

    print(f"\nAnálisis de la columna '{columna}':")
    print(f"Total de registros: {total_registros}")

    for key in analisis.keys():
        if key != 'total_registros':
            porcentaje = (analisis[key] / total_registros) * 100
            print(f"{key}: {analisis[key]} registros ({porcentaje:.2f}%)")

    print("\nEjemplos de registros problemáticos:")
    for tipo, ejemplos in patrones.items():
        if ejemplos:
            print(f"\n{tipo}:")
            for ejemplo in ejemplos:
                print(f"  - {ejemplo}")

    return analisis, patrones

**Función para limpiar la columna COLONIA**

In [ ]:
def limpiar_colonia(df, columna='COLONIA'):
    # Convertir a string y crear copia para análisis
    serie = df[columna].astype(str)

    # Función interna para limpiar un valor individual
    def limpiar_valor(valor):
        # Manejar valores nulos o vacíos
        if pd.isna(valor) or valor.strip() == '' or valor == 'nan':
            return ''

        # Si el valor contiene solo números (incluso con espacios), devolver cadena vacía
        if valor.replace(' ', '').isdigit():
            return ''

        # Eliminar espacio inicial y final
        valor = valor.strip()

        # Eliminar punto al inicio
        if valor.startswith('.'):
            valor = valor[1:]

        # Eliminar '+' o '-' al inicio
        if valor.startswith('+'):
            valor = valor[1:]

        if valor.startswith('-'):
            valor = valor[1:]

        # Eliminar todos los paréntesis
        valor = valor.replace('(', '').replace(')', '')

        # Eliminar signos al final: '+', '-', ':', '.'
        valor = re.sub(r'[+\-:.]$', '', valor)

        # Eliminar dobles espacios
        valor = re.sub(r'\s+', ' ', valor)

        # Convertir a mayúsculas
        valor = valor.upper()

        return valor.strip()

    # Aplicar limpieza a toda la serie
    serie_limpia = serie.apply(limpiar_valor)

    # Crear nuevo DataFrame con la columna limpia
    df_limpio = df.copy()
    df_limpio[columna] = serie_limpia

    return df_limpio

##***Columna del la Poblacion (POBL)***

**Función para revisar la columna**

In [30]:
def analizar_columna_pobl(df, columna='POBL'):

    def _verificar_formato(valor):
        if pd.isna(valor):
            return ''

        # Convertir a string para asegurar consistencia
        valor = str(valor)

        # Verificar cantidad de números al inicio
        numeros_inicio = len(re.match(r'^\d+', valor).group(0)) if re.match(r'^\d+', valor) else 0

        # Verificar espacios
        tiene_espacios_extra = valor != valor.strip()
        tiene_multiples_espacios = '  ' in valor

        # Verificar mayúsculas
        esta_en_mayusculas = valor.isupper()

        # Verificar caracteres especiales
        tiene_caracteres_especiales = bool(re.search(r'[^A-Z0-9\s]', valor))

        return {
            'original': valor,
            'numeros_inicio': numeros_inicio,
            'espacios_extra': tiene_espacios_extra,
            'multiples_espacios': tiene_multiples_espacios,
            'mayusculas': esta_en_mayusculas,
            'caracteres_especiales': tiene_caracteres_especiales
        }

    # Aplicar análisis a toda la columna
    analisis = df[columna].apply(_verificar_formato)

    # Resumen estadístico
    resumen = {
        'total_registros': len(df),
        'nulos': sum(1 for item in analisis if item == 'NaN'),
        'mas_de_3_numeros_inicio': sum(1 for item in analisis if item != 'NaN' and item['numeros_inicio'] > 3),
        'con_espacios_extra': sum(1 for item in analisis if item != 'NaN' and item['espacios_extra']),
        'con_multiples_espacios': sum(1 for item in analisis if item != 'NaN' and item['multiples_espacios']),
        'en_minusculas': sum(1 for item in analisis if item != 'NaN' and not item['mayusculas']),
        'con_caracteres_especiales': sum(1 for item in analisis if item != 'NaN' and item['caracteres_especiales'])
    }

    return {
        'resumen': resumen,
        'detalles': analisis
    }

**Función para limpiar la columna POBL**

In [29]:
def limpiar_columna_pobl(df, columna='POBL'):
    def _limpiar_pobl(valor):
        # Manejar valores nulos
        if pd.isna(valor):
            return ''

        # Convertir a string
        valor = str(valor)

        # Convertir a mayúsculas
        valor = valor.upper()

        # Eliminar espacios al inicio y al final
        valor = valor.strip()

        # Eliminar caracteres especiales
        valor = re.sub(r'[^A-Z0-9\s]', '', valor.upper())

        # Reemplazar múltiples espacios por un solo espacio
        valor = re.sub(r'\s+', ' ', valor)

        # Verificar formato: primeros 3 caracteres deben ser números
        match = re.match(r'^(\d{3})\s*(.*)$', valor)

        # Si no cumple el formato, dejar en blanco
        if not match:
            return ''

        # Formatear: 3 números, espacio, resto en mayúsculas
        numeros, nombre = match.groups()
        return f"{numeros} {nombre.strip()}"

    # Crear una copia del DataFrame para no modificar el original
    df_limpio = df.copy()

    # Aplicar limpieza
    df_limpio[columna] = df_limpio[columna].apply(_limpiar_pobl)

    return df_limpio

# CODIGOS DE LIMPIEZA
CARLOS

In [ ]:
##CARLOS

# CODIGOS DE LIMPIEZA
RICARDO

In [ ]:
##RICARDO

In [ ]:
# Crear listas para almacenar datos inválidos
errores_paterno = []
errores_materno = []

# Expresión regular para validar valores tipo "S.A. DE C.V." correctamente
pattern = r"^(?:[A-Z]+(?:\.[A-Z]+)*)(?: [A-Z]+(?:\.[A-Z]+)*)*$"

# Validar y limpiar las columnas APE_PATERNO y APE_MATERNO
for col in ['AP_PATERNO', 'AP_MATERNO']:
    datos_invalidos = []
    df_principal[col] = df_principal[col].apply(
        lambda x: (
            # Si el valor es vacío por defecto, mantenerlo vacío
           "" if pd.isnull(x) or (isinstance(x, str) and x.strip() == "")  # Manejo de NaN y valores vacíos
            # Limpieza básica y validación
            else ' '.join(x.strip().upper().split())  # Quitar espacios al inicio, final y dobles
            if isinstance(x, str) and (
                re.fullmatch(pattern, x.strip()) or  # Patrón para letras con puntos válidos
                x.replace(' ', '').isalpha()        # Letras sin puntos, solo con espacios
            )
            else "" if isinstance(x, str) and not x.strip().isalnum()  # Vacíos o caracteres solos
            else datos_invalidos.append(x) or x  # Si no cumple, mantener el valor original y registrar
        )
    )
    # Guardar los datos inválidos para mostrarlos después
    if col == 'AP_PATERNO':
        errores_paterno = datos_invalidos
    else:
        errores_materno = datos_invalidos

# Mostrar resultados
print("\nPrimeras filas del DataFrame después de la limpieza:")
print(df_principal.head())

# Mostrar datos inválidos para APE_PATERNO
if errores_paterno:
    print("\nDatos inválidos en 'AP_PATERNO':")
    print(errores_paterno)

# Mostrar datos inválidos para APE_MATERNO
if errores_materno:
    print("\nDatos inválidos en 'AP_MATERNO':")
    print(errores_materno)



Primeras filas del DataFrame después de la limpieza:
                                  ASEG              SERIE    TELEFONO  \
0              REY DAVID BELMAN LOZANO  9BWKL45U6RP023988  4444191129   
1                  DAVILA MORENO CELSO  3G1TA5AF7HL120346  5560767580   
2        GARCIA GONZALEZ JESUS EDUARDO  1C4GJ25R23B244434  4611366507   
3  SD INDEVAL INSTITUCION PARA EL DEPO  JM3KKKHC2R1166032         NaN   
4           MARIA SEDELIA VARGAS ROSAS  MB2C22AC8JM241475  7531117734   

           DIR_ASEG_CALLE             COLONIA  \
0      CIRCUITO JACARANDA           EL ZAPOTE   
1  1RA CDA FERROCARRIL 12  SAN JUAN TEMAMATLA   
2    PROLONGACION CONSEJO             METEPEC   
3     PASEO DE LA REFORMA          CUAUHTEMOC   
4       LOC EL PUERTECITO           EL REPARO   

                                   POBL            ESTADO       CP  \
0       035 SOLEDAD DE GRACIANO SANCHEZ   SAN LUIS POTOSI  78430.0   
1                         083 TEMAMATLA  ESTADO DE MEXICO  56650.0   
2 

In [ ]:
#import pandas as pd

# Supongamos que ya cargaste tu DataFrame en 'df_principal'
# df_principal = pd.read_csv('tu_archivo.csv')

# Verificar la columna 'RFC_Aseg'
df_principal['RFC_ASEG'] = df_principal['RFC_ASEG'].apply(
    lambda rfc: rfc.strip().upper() if isinstance(rfc, str) and rfc.strip().isalnum() else None
)

# Mostrar algunas filas del DataFrame para verificar resultados
print("Primeras filas después de la limpieza:")
print(df_principal.head())

# Mostrar filas con errores
errores = df_principal[df_principal['RFC_ASEG'].isnull()]
if not errores.empty:
    print("\nFilas con errores:")
    print(errores)

# Guardar el resultado en un archivo si lo necesitas
#df_principal.to_csv('df_principal_limpio.csv', index=False)
#print("\nArchivo limpio guardado como 'df_principal_limpio.csv'")


Primeras filas después de la limpieza:
                                  ASEG              SERIE    TELEFONO  \
0              REY DAVID BELMAN LOZANO  9BWKL45U6RP023988  4444191129   
1                  DAVILA MORENO CELSO  3G1TA5AF7HL120346  5560767580   
2        GARCIA GONZALEZ JESUS EDUARDO  1C4GJ25R23B244434  4611366507   
3  SD INDEVAL INSTITUCION PARA EL DEPO  JM3KKKHC2R1166032         NaN   
4           MARIA SEDELIA VARGAS ROSAS  MB2C22AC8JM241475  7531117734   

           DIR_ASEG_CALLE             COLONIA  \
0      CIRCUITO JACARANDA           EL ZAPOTE   
1  1RA CDA FERROCARRIL 12  SAN JUAN TEMAMATLA   
2    PROLONGACION CONSEJO             METEPEC   
3     PASEO DE LA REFORMA          CUAUHTEMOC   
4       LOC EL PUERTECITO           EL REPARO   

                                   POBL            ESTADO       CP  \
0       035 SOLEDAD DE GRACIANO SANCHEZ   SAN LUIS POTOSI  78430.0   
1                         083 TEMAMATLA  ESTADO DE MEXICO  56650.0   
2                

# CODIGOS DE LIMPIEZA
YESS

In [ ]:
##YESS

In [ ]:
#LIMPIEZA CP_CONTRATANTE
import re

def limpiar_cp(cp):
    if pd.isna(cp) or cp == "":  # Verificar si el valor está vacío o es NaN
        return cp
    cp = str(cp).strip().replace(" ", "")  # Eliminar espacios al principio, al final y entre los números
    if cp.isdigit() and (len(cp) == 4 or len(cp) == 5):  # Validar si es numérico y tiene 4 o 5 dígitos
        return cp
    else:
        print(f"Valor no válido encontrado: {cp}")  # Mostrar los valores que no cumplen las condiciones
        return cp  # Dejar el valor tal como está si no cumple las condiciones

# Aplicar la limpieza a la columna
df_principal['CP_CONTRATANTE'] = df_principal['CP_CONTRATANTE'].apply(limpiar_cp)

# Mostrar las primeras 10 filas con posibles problemas de validación
df_invalidos = df_principal[~df_principal['CP_CONTRATANTE'].str.match(r'^\d{4,5}$', na=True)]
print(df_invalidos.head(10))

Empty DataFrame
Columns: [ASEG, SERIE, TELEFONO, DIR_ASEG_CALLE, COLONIA, POBL, ESTADO, CP, CORREO, NOMBRE, AP_PATERNO, AP_MATERNO, NUM_INTERNO, NUM_EXTERNO, RFC_ASEG, RFC_CONTRATANTE, CP_CONTRATANTE, DOMICILIO_FISCAL, BENEF_PREFERENTE, ASEGURADO, TIPO_PERSONA, FEC_NACIMIENTO, es_valido]
Index: []

[0 rows x 23 columns]


In [ ]:
df_principal.head(20)

,ASEG,SERIE,TELEFONO,DIR_ASEG_CALLE,COLONIA,POBL,ESTADO,CP,CORREO,NOMBRE,...,NUM_INTERNO,NUM_EXTERNO,RFC_ASEG,RFC_CONTRATANTE,CP_CONTRATANTE,DOMICILIO_FISCAL,BENEF_PREFERENTE,ASEGURADO,TIPO_PERSONA,FEC_NACIMIENTO
0,REY DAVID BELMAN LOZANO,9BWKL45U6RP023988,4444191129,CIRCUITO JACARANDA,EL ZAPOTE,035 SOLEDAD DE GRACIANO SANCHEZ,SAN LUIS POTOSI,78430.0,SOPORTE@ODESSA.COM,REY DAVID,...,NaN,216,BELR771229I82,NaN,NaN,NaN,NaN,29588257,1 Fisica,29/12/77
1,DAVILA MORENO CELSO,3G1TA5AF7HL120346,5560767580,1RA CDA FERROCARRIL 12,SAN JUAN TEMAMATLA,083 TEMAMATLA,ESTADO DE MEXICO,56650.0,CELSO0605@GMAIL.COM,CELSO,...,NaN,.,DAMC560506,NaN,NaN,NaN,NaN,23005894,1 Fisica,06/05/56
2,GARCIA GONZALEZ JESUS EDUARDO,1C4GJ25R23B244434,4611366507,PROLONGACION CONSEJO,METEPEC,019 ATLIXCO,PUEBLA,74360.0,JESUSEDUARDOGARCIA10@GMAIL.COM,JESUS EDUARDO,...,NaN,1,GAGJ721128,NaN,NaN,NaN,NaN,26706560,1 Fisica,28/11/72
3,SD INDEVAL INSTITUCION PARA EL DEPO,JM3KKKHC2R1166032,NaN,PASEO DE LA REFORMA,CUAUHTEMOC,015 CUAUHTEMOC,CIUDAD DE MEXICO,6500.0,.,SD INDEVAL INSTITUCION PARA EL DEPOSITO DE VALORE,...,PIS,255,SDI870821SXA,NaN,NaN,NaN,NaN,24785634,2 Moral,01/01/90
4,MARIA SEDELIA VARGAS ROSAS,MB2C22AC8JM241475,7531117734,LOC EL PUERTECITO,EL REPARO,068 LA UNION DE ISIDORO MONTES DE OC,GUERRERO,40803.0,NaN,MARIA SEDELIA,...,NaN,1,VARS630604,NaN,NaN,NaN,NaN,29853119,1 Fisica,04/06/63
5,QUALTIA ALIMENTOS OPERACIONES S DE,19XFC2649KE601456,(81) 8040-660,AVE. CONDUCTORES NO. 600,LA GRANGE,046 SAN NICOLAS DE LOS GARZA,NUEVO LEON,66480.0,NaN,QUALTIA ALIMENTOS OPERACIONES S DE RL DE CV,...,2,200,QAO680613E91,NaN,NaN,NaN,NaN,29574113,2 Moral,01/01/90
6,REYNA CORONADO GILBERTO,93CXM19226C136575,NaN,GIRASOL,PASEO DEL NOGAL-066004,018 GARCIA,NUEVO LEON,66004.0,NaN,GILBERTO,...,NaN,642,RECG720425,NaN,NaN,NaN,NaN,31171800,1 Fisica,25/04/72
7,ARRENDADORA DE AUTOS MATAMOROS S.A,MMBMLV5GXNH007116,NaN,JUAREZ,FRACC PORTAL SAN MIGUEL,032 REYNOSA,TAMAULIPAS,88500.0,.,ARRENDADORA DE AUTOS MATAMOROS S.A DE C.V.,...,NaN,1050,AAM850305830,NaN,NaN,NaN,NaN,6294289,2 Moral,01/01/90
8,PARAL DISTRIBUIDORA DE REFRESCOS S.,JALC4B16387006487,9611181648,LIBRAMIENTO SUR ORIENTE,COLONIAL,101 TUXTLA GUTIERREZ,CHIAPAS,29098.0,JJSANMON@GPARAL.COM.MX,PARAL DISTRIBUIDORA DE REFRESCOS S.A. DE C.V.,...,NaN,3851,PDR011126692,NaN,30870.0,NaN,NaN,25943674,2 Moral,01/01/90
9,MARINELY MENDEZ REVILLA,3N1CB51S74L087671,.,CARRETERA PICACHO AJUSCO,JARDINES EN LA MONTA A,012 TLALPAN,CIUDAD DE MEXICO,14210.0,.,MARINELY,...,NaN,170,MERM7808159XA,NaN,NaN,NaN,NaN,21109175,1 Fisica,15/08/78


# GUARDAR ARCHIVO MODIFICADO

In [ ]:
# Ruta para guardar el archivo consolidado
output_csv_path = '/content/drive/My Drive/Qualitas/Dataset.csv'

# Guardar el DataFrame final
df_principal.to_csv(output_csv_path, index=False)

print(f"Archivo consolidado guardado en: {output_csv_path}")


Archivo consolidado guardado en: /content/drive/My Drive/Qualitas/Dataset.csv
